# Using EcoFOCIpy to process raw field data

## SKQ202412S (Arctic - Sikuliaq)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [88]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [89]:
sample_data_dir = '/Users/bell/ecoraid/2024/CTDcasts/skq202412s/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [90]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'SKQ202412S' #no hyphens
cruise_meta_file = sample_data_dir+'logs/SKQ202412S.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd014.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd015.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd016.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd029.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd030.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd031.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd032.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd033.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd034.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd035.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd036.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd037.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd038.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd039.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd040.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd042.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd043.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd044.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd045.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd046.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd047.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd048.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd049.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd053.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd054.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd055.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd057.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd059.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd061.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd063.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd064.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd066.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd067.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd068.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd070.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd072.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd074.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd076.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/skq202412s/rawconverted/ctd079.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [91]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,depsm,latitude,longitude,prdm,t090c,t190c,potemp090c,sal00,sal11,density00,sigma-00,sbox0mm/kg,oxsolmm/kg,sbeox0ps,v0,fleco-afl,cstarat0,scan,datetime
count,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20
mean,32.52760,62.195144,-174.676072,32.846700,2.577735,2.563570,2.576670,31.438975,31.438655,1025.146465,24.989445,305.836000,337.371689,91.329850,0.043230,0.816865,0.492455,23305.900000,2024-08-07 17:29:13.650000128
min,2.62100,62.195000,-174.676950,2.647000,-1.085900,-1.085900,-1.086900,30.926500,31.053400,1024.106500,24.094400,233.908000,296.191620,64.735000,0.020600,0.275000,0.274400,8693.000000,2024-08-07 17:19:05
25%,10.47000,62.195020,-174.676900,10.571750,-0.551675,-0.551650,-0.553475,31.022525,31.070675,1024.213050,24.169425,283.611500,296.364407,77.223500,0.025575,0.393400,0.312325,14280.250000,2024-08-07 17:22:57.750000128
50%,29.00150,62.195130,-174.676190,29.284500,0.269300,0.268750,0.268350,31.402250,31.445700,1025.321400,25.185850,291.230000,355.308585,98.242500,0.041700,0.781150,0.348400,23515.000000,2024-08-07 17:29:22.500000
75%,50.43925,62.195260,-174.675238,50.934250,7.872050,7.872125,7.871750,31.762600,31.651175,1025.777925,25.532400,338.991500,361.331865,104.772000,0.051125,1.005800,0.536525,33209.750000,2024-08-07 17:36:06.500000
max,67.99700,62.195320,-174.674940,68.667000,7.876100,7.877300,7.875100,32.025400,32.003500,1026.057000,25.727800,384.615000,367.266060,108.449000,0.084500,1.807200,1.060700,37857.000000,2024-08-07 17:39:20
std,23.20137,0.000123,0.000815,23.430211,3.893949,3.889677,3.894189,0.391522,0.327362,0.740154,0.631601,50.454875,30.627325,16.108364,0.019019,0.456317,0.296870,10378.975484,NaN


## Create BTL report file

In [92]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [93]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'SKQ202412S',
 'CruiseID_Historic': 'SKQ202412S',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'Sikuliaq',
 'ShipID': 'SK',
 'StartDate': datetime.date(2023, 8, 6),
 'EndDate': datetime.date(2024, 8, 28),
 'Project': 'UAF,FOCI,DBO,AMBON',
 'ChiefScientist': 'Seth Danielson',
 'StartPort': 'Nome, AK',
 'EndPort': 'Nome, AK',
 'CruiseLocation': 'Arctic',
 'Description': 'Arctic Mooring Cruise',
 'CruiseYear': 2024,
 'ctdlogs_pdf_name': 'SKQ202412S_CastLogs.pdf'}

In [94]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54720,
 'Vessel': 'R/V Sikuliaq',
 'CruiseID': 'SKQ202412S',
 'Project_Leg': '',
 'UniqueCruiseID': 'SKQ202412S',
 'Project': 'Arctic Collaborative Ecosystem Cruise',
 'StationNo_altname': '',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 62,
 'LatitudeMin': 11.73,
 'LongitudeDeg': 174,
 'LongitudeMin': 40.42,
 'GMTDay': 7,
 'GMTMonth': 'Aug',
 'GMTYear': 2024,
 'GMTTime': 61975,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': '',
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': -9999,
 'StationNameID': '',
 'MaxDepth': -9999,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'WaterMassCode':

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [95]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [96]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag',
                        'latitude':'latitude_btl',
                        'longitude':'longitude_btl'})

cruise_data['ctd001.btl'].sample()

,depsm,latitude_btl,longitude_btl,Pressure [dbar],temperature_ch1,temperature_ch2,potemp090c,salinity_ch1,salinity_ch2,density00,sigma-00,oxy_conc_ch1,oxsolmm/kg,oxy_percentsat_ch1,v0,chlor_fluorescence,Attenuation,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,,
8.0,40.668,62.19522,-174.67558,41.066,-0.8519,-0.8583,-0.8528,31.601,31.5544,1025.5923,25.3946,335.698,365.441,91.861,0.037,0.6673,0.2868,18203.0,2024-08-07 17:25:41,ctd001


In [97]:
cruise_data['ctd001.btl'].columns

Index(['depsm', 'latitude_btl', 'longitude_btl', 'Pressure [dbar]',
       'temperature_ch1', 'temperature_ch2', 'potemp090c', 'salinity_ch1',
       'salinity_ch2', 'density00', 'sigma-00', 'oxy_conc_ch1', 'oxsolmm/kg',
       'oxy_percentsat_ch1', 'v0', 'chlor_fluorescence', 'Attenuation', 'scan',
       'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [98]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [99]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [100]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 20)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 17.0 18.0 19.0 20.0
Data variables: (12/20)
    depsm               (bottle) float64 68.0 67.89 67.58 ... 2.621 2.933 2.934
    latitude_btl        (bottle) float64 62.2 62.2 62.2 62.2 ... 62.2 62.2 62.2
    longitude_btl       (bottle) float64 -174.7 -174.7 -174.7 ... -174.7 -174.7
    Pressure [dbar]     (bottle) float64 68.67 68.56 68.25 ... 2.647 2.962 2.962
    temperature_ch1     (bottle) float64 -0.5516 -0.5506 -0.5519 ... 7.875 7.873
    temperature_ch2     (bottle) float64 -0.5505 -0.55 -0.5516 ... 7.875 7.875
    ...                  ...
    v0                  (bottle) float64 0.0213 0.0206 0.0221 ... 0.0412 0.0401
    chlor_fluorescence  (bottle) float64 0.29 0.275 0.3109 ... 0.7689 0.7414
    Attenuation         (bottle) float64 1.054 1.051 1.061 ... 0.3503 0.3507
    scan                (bottle) float64 8.693e+03 8.876e+03 ... 3.786e+04
    datetime            (bottle) datetime64[ns] 2024-08-07T17:19:05 ... 2024-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [101]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [102]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [103]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 20, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 17.0 18.0 19.0 20.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/20)
    depsm               (latitude, longitude, time, bottle) float64 68.0 ... ...
    latitude_btl        (latitude, longitude, time, bottle) float64 62.2 ... ...
    longitude_btl       (latitude, longitude, time, bottle) float64 -174.7 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 68.67 ......
    temperature_ch1     (latitude, longitude, time, bottle) float64 -0.5516 ....
    temperature_ch2     (latitude, longitude, time, bottle) float64 -0.5505 ....
    ...                  ...
    v0                  (latitude, longitude, time, bottle) float64 0.0213 .....
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 0.29 ... ...
    Attenuation         (latitude, longitude, time, bottle) float64 1.054 ......
    scan                (latitude, longitude, time, bottle) float64 8.693e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [104]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 20, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 17.0 18.0 19.0 20.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/20)
    depsm               (latitude, longitude, time, bottle) float64 68.0 ... ...
    latitude_btl        (latitude, longitude, time, bottle) float64 62.2 ... ...
    longitude_btl       (latitude, longitude, time, bottle) float64 -174.7 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 68.67 ......
    temperature_ch1     (latitude, longitude, time, bottle) float64 -0.5516 ....
    temperature_ch2     (latitude, longitude, time, bottle) float64 -0.5505 ....
    ...                  ...
    v0                  (latitude, longitude, time, bottle) float64 0.0213 .....
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 0.29 ... ...
    Attenuation         (latitude, longitude, time, bottle) float64 1.054 ......
    scan                (latitude, longitude, time, bottle) float64 8.693e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      SKQ202412S
    CruiseID_Historic:             SKQ202412S
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        Sikuliaq
    ShipID:                        SK
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-01-06T08:19:17Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [112]:
cruise_data[cast] = cruise_data[cast].rename(columns={
                    't090c':'temperature_ch1',
                    't190c':'temperature_ch2',
                    'sal00':'salinity_ch1',
                    'sal11':'salinity_ch2',
                    'sbox0mm/kg':'oxy_conc_ch1',
                    'sbeox0ml/l':'oxy_concM_ch1',
                    'sbox1mm/kg':'oxy_conc_ch2',
                    'sbeox1ml/l':'oxy_concM_ch2',
                    'sbeox0ps':'oxy_percentsat_ch1',
                    'sbeox1ps':'oxy_percentsat_ch2',
                    'OxsolMm/Kg':'oxy_sol_ch1',
                    'oxsolmm/kg':'oxy_sol_ch1',
                    'sigma-t00':'sigma_t_ch1',
                    'sigma-t11':'sigma_t_ch2',
                    'cstarat0':'Attenuation',
                    'cstartr0':'Transmittance',
                    'fleco-afl':'chlor_fluorescence',
                    'turbwetntu0':'turbidity',
                    'prdm':'pressure',
                    'empty':'empty', #this will be ignored
                    'flag':'flag',
                    'latitude':'latitude_btl',
                    'longitude':'longitude_btl'})

cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                            instrument_yaml=inst_config, 
                            operation_yaml=cruise_config,
                            operation_type='ctd')

cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

#add global attributes
cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cast_label = cast.lower().split('d')[-1].split('.')[0]
cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                           filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


In [113]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'OxsolMm/Kg':'oxy_sol_ch1',
                            'oxsolmm/kg':'oxy_sol_ch1',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag',
                            'latitude':'latitude_btl',
                            'longitude':'longitude_btl'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history